In [1]:
import h2o
from pandas import DataFrame as DataFrame
from pandas import concat as concat
from collections import OrderedDict as OrderedDict
import json
import numpy as np

In [2]:
import os
import sys
import tarfile
from six.moves import cPickle as pickle
# In[2]:

os.chdir('d:/Data/Gdeeplearning-Udacity')
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['valid_dataset']
  train_labels = save['valid_labels']
  valid_dataset = save['test_dataset']
  valid_labels = save['test_labels']

  del save  # hint to help gc free up memory
    
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)


Training set (10000, 28, 28) (10000,)
Validation set (10000, 28, 28) (10000,)


In [3]:
# In[5]:

image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = DataFrame(dataset.reshape((-1, image_size * image_size)).astype(np.float32))
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    columns = list()
    for each_value in dataset.columns.values:
        columns.append(str(each_value))
    dataset.columns = columns
    labels = DataFrame(labels.reshape((-1, 1)).astype(str))
    columns = list()
    for each_value in labels.columns.values:
        columns.append('objective'+ str(each_value))
    labels.columns = columns
    #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

pd_train_dataset, pd_train_labels = reformat(train_dataset, train_labels)
pd_valid_dataset, pd_valid_labels = reformat(valid_dataset, valid_labels)
pd_train_dataset = concat([pd_train_dataset,pd_train_labels],axis=1)
pd_valid_dataset = concat([pd_valid_dataset,pd_valid_labels],axis=1)
                       
print('Training set', pd_train_dataset.shape)
print('Validation set', pd_valid_dataset.shape)

# In[9]:

#deleting non used memory
#train_dataset = []
#train_labels = []
#valid_dataset = []
#valid_labels = []


Training set (10000, 785)
Validation set (10000, 785)


In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

from h2o import save_model as save_model
from h2o import load_model as load_model
from h2o import init as init
from h2o import connect as connect
from h2o import cluster as cluster
from h2o.model.metrics_base import H2OBinomialModelMetrics as H2OBinomialModelMetrics
from h2o.model.metrics_base import H2OMultinomialModelMetrics as H2OMultinomialModelMetrics
from h2o.model.metrics_base import H2ORegressionModelMetrics as H2ORegressionModelMetrics

from h2o import H2OFrame as H2OFrame
from pandas import DataFrame as DataFrame
import json
import time
from random import randint


class H2OHandler(object):
    """
    H2OHandler

    Train: Get some analysis list of tuplas (analysis_results.json +  (algorithm + normalzations over a Dataframe) and launch de analysis on H2O platform
    predict: Get some list of [analysis_results.json] and load and execute algorithm
    Algorithms and model operation:

    "H2OGradientBoostingEstimator" :{
      "id" : "number",
      "module" : "h2o.estimators.gbm",
      "types": {
        "binomial": {"active" : true, "valued": "enum"},
        "multinomial": {"active" : true, "valued": "enum"},
        "regression" : {"active" : true, "valued": "float64"},
        "topology" : {"active" : false, "valued": "float64"}
      }

    "H2OGeneralizedLinearEstimator" :{
      "id" : "number",
      "module" : "h2o.estimators.glm",
      "types": {
        "binomial": {"active" : true, "valued": "enum"},
        "multinomial": {"active" : true, "valued": "enum"},
        "regression" : {"active" : true, "valued": "float64"},
        "topology" : {"active" : false, "valued": "float64"}
      }

    "H2ODeepLearningEstimator" :{
      "id" : "number",
      "module" : "h2o.estimators.deeplearning",
      "types": {
        "binomial": {"active" : true, "valued": "enum"},
        "multinomial": {"active" : true, "valued": "enum"},
        "regression" : {"active" : true, "valued": "float64"},
        "topology" : {"active" : false, "valued": "float64"}
      }

    "H2ORandomForestEstimator" :{
      "id" : "number",
      "module" : "h2o.estimators.random_forest",
      "types": {
        "types": {
          "binomial": {"active" : true, "valued": "enum"},
          "multinomial": {"active" : true, "valued": "enum"},
          "regression" : {"active" : true, "valued": "float64"},
          "topology" : {"active" : false, "valued": "float64"}
        }

    Status Codes:
        -1 : uninitialized
        0: Success

    """

    def __init__(self):
        self._model_base = None
        self.path = r'D:\\Data\\models'
        self.url = 'http://127.0.0.1:54321'
        self.nthreads = 6
        self.ice_root = r'D:/Data/h2o-logs'
        self.max_mem_size = '8G'
        self.start_h2o = True
        
        try:
            connect(url=self.url)
        except:
            init(url=self.url, nthreads=self.nthreads, ice_root=self.ice_root, max_mem_size=self.max_mem_size) 
    
    @classmethod
    def shutdown_cluster(cls):       
        try:
            cluster().shutdown()
        except:
            print('H20-cluster not working')

    def order_training(self, analysis_id, training_frame, valid_frame, analysis_list):
        assert isinstance(analysis_id, str)
        assert isinstance(training_frame, DataFrame)
        assert isinstance(valid_frame, DataFrame) or valid_frame is None
        assert isinstance(analysis_list, list)

        analysis_id = analysis_id  # "customer_" + #random + #timestamp
        status = -1  # Operation Code
        analysis_list = analysis_list  # [(ar(incomplete).json, normalizations_sets.json)]
        model_list = list()
        analysis_timestamp = str(time.time())
        type_analysis = None

        training_frame = H2OFrame(python_obj=training_frame)
        if valid_frame is not None:
            valid_frame = H2OFrame(python_obj=valid_frame)

        for algorithm_description, normalization in analysis_list:
            # Mapping

            struct_ar = json.load(algorithm_description, object_pairs_hook=OrderedDict)
            print(struct_ar)

            if normalization is not None:
                struct_ns = json.load(algorithm_description, object_pairs_hook=OrderedDict)
            else:
                struct_ns = None

            assert isinstance(struct_ar, OrderedDict)
            assert isinstance(struct_ns, OrderedDict) or normalization is None

            for each_model in struct_ar['model_parameters']['h2o']:
                model_timestamp = str(time.time())

                y = each_model['parameters']['response_column']['value']
                x = training_frame.col_names
                x.remove(y)

                if each_model['types'][0]['active']:
                    if each_model['types'][0]['type'] in ['binomial', 'multinomial']:
                        training_frame[y] = training_frame[y].asfactor()
                        if valid_frame is not None:
                            valid_frame[y] = valid_frame[y].asfactor()
                    type_analysis = each_model['types'][0]['type']

                model_command = list()
                model_command.append(each_model['module'])
                model_command.append(".")
                model_command.append(each_model['model'])
                model_command.append("(")
                model_command.append("training_frame=training_frame")

                if valid_frame is not None:
                    model_command.append(", validation_frame=valid_frame")
                
                model_command.append(", model_id='%s'" % (each_model['model'] + '_' + model_timestamp))
                for key, value in each_model['parameters'].items():
                    if value['seleccionable']:
                        if isinstance(value['value'], str):
                            model_command.append(", %s=\'%s\'" % (key, value['value']))
                        else:
                            model_command.append(", %s=%s" % (key, value['value']))

                model_command.append(")")
                model_command = ''.join(model_command)
                print(model_command)
                self._model_base = eval(model_command)

                if valid_frame is not None:
                    self._model_base.train(x=x, y=y, training_frame=training_frame, validation_frame=valid_frame)
                else:
                    self._model_base.train(x=x, y=y, training_frame=training_frame)

                load_path = list()
                load_path.append(self.path)
                load_path.append('/')
                load_path.append(struct_ar['model_id'])
                load_path.append('/')
                load_path.append('train')

                save_model(model=self._model_base, path=''.join(load_path), force=True)
                # Generate json ar.json
                final_ar_model = struct_ar.copy()
                final_ar_model['type'] = 'train'
                final_ar_model['timestamp'] = analysis_timestamp
                final_ar_model['load_path'] = ''.join(load_path) + each_model['model'] + '_' + model_timestamp
                final_ar_model['metrics'] = self._generate_metrics(training_frame, type_analysis)
                final_ar_model['model_parameters']['h2o'] = []
                final_ar_model['model_parameters']['h2o'] = [each_model.copy()]
                final_ar_model['model_parameters']['h2o'][0]['parameters']['model_id'] = \
                    each_model['model'] + '_' + model_timestamp
                final_ar_model['metrics'] = OrderedDict()
                final_ar_model['metrics']['train'] = self._generate_metrics(dataframe=None, source='train')
                if valid_frame is not None:
                    final_ar_model['metrics']['valid'] = self._generate_metrics(dataframe=None, source='valid')
                final_ar_model['metrics']['xval'] = self._generate_metrics(dataframe=None, source='xval')
                model_list.append(final_ar_model)
        return analysis_id, model_list.copy()

    def predict(self, dataframe, algorithm_description):
        type_ = None
        model_timestamp = str(time.time())
        type_analysis = None

        struct_ar = json.load(algorithm_description, object_pairs_hook=OrderedDict)
        try:
            print(struct_ar['load_path']['value'])
            self._model_base = load_model(struct_ar['load_path']['value'])
        except:
            return ('Necesario cargar un modelo valid o ar.json valido')

        predict_frame = H2OFrame(python_obj=dataframe)

        y = struct_ar['model_parameters']['h2o'][0]['parameters']['response_column']['value']
        if struct_ar['model_parameters']['h2o'][0]['types'][0]['type'] in ['binomial', 'multinomial']:
            predict_frame[y].asfactor()

        struct_ar['type'] = 'predict'
        struct_ar['timestamp'] = model_timestamp
        struct_ar['metrics'] = OrderedDict()
        struct_ar['metrics']['predict'] = self._generate_metrics(predict_frame, source=None)

        return (self._model_base.predict(predict_frame).as_data_frame(use_pandas=True),
                struct_ar.copy())

    def _generate_metrics(self, dataframe, source):
        """
        Generate model metrics for this model on test_data.

        :param H2OFrame test_data: Data set for which model metrics shall be computed against. All three of train,
            valid and xval arguments are ignored if test_data is not None.
        :param bool train: Report the training metrics for the model.
        :param bool valid: Report the validation metrics for the model.
        :param bool xval: Report the cross-validation metrics for the model. If train and valid are True, then it
            defaults to True.
        """

        model_metrics = OrderedDict()
        
        if dataframe is not None:
            perf_metrics = self._model_base.model_performance(dataframe)
        else:
            if source == 'valid':
                perf_metrics = self._model_base.model_performance(valid=True)
            elif source == 'xval':
                perf_metrics = self._model_base.model_performance(xval=True)
            else:
                perf_metrics = self._model_base.model_performance(train=True)
        
        for parameter, value in perf_metrics._metric_json.items():
            if parameter in ['hit_ratio_table','gains_lift']:
                model_metrics[parameter] = value.as_data_frame()
            elif parameter == 'cm':
                model_metrics[parameter] = value['table'].as_data_frame()            
            elif not isinstance(value, dict) and parameter not in ['model_checksum', 'frame_checksum', 'description']:
                model_metrics[parameter] = value
                
        return model_metrics.copy()                        

    def get_metric(self, algorithm_description, metric, source): #not tested
        try:
            struct_ar = OrderedDict(json.load(algorithm_description))
            model_metrics = struct_ar['metrics']['source']
        except:
            return ('Necesario cargar un modelo valid o ar.json valido')
        try:
            return struct_ar['metrics']['source'][metric]
        except:
            return 'Not Found'

In [ ]:
init(url='http://127.0.0.1:54321', nthreads=6, ice_root=r'D:/Data/h2o-logs', max_mem_size='4G' )

In [ ]:
json_file = open(r'D:\e2its-dayf.svn\gdayf\branches\0.0.2-jlsanchez\test\json\algorithm_result(ar)-prueba-carga.json')
analysis_list =  [(json_file, None)]

In [ ]:
json_file = open(r'D:\e2its-dayf.svn\gdayf\branches\0.0.2-jlsanchez\test\json\algorithm_result(ar)-2.json')
analysis_list.append((json_file, None))

In [ ]:
analysis_models = H2OHandler()
analysis_results = analysis_models.order_training(analysis_id='PoC_1',  training_frame=pd_train_dataset, 
                                  valid_frame=pd_train_dataset, analysis_list=analysis_list )


In [ ]:
print(analysis_results)

In [ ]:
json_file = open(r'D:\e2its-dayf.svn\gdayf\branches\0.0.2-jlsanchez\test\json\algorithm_result(ar)-2.json')

analysis_models = H2OHandler()
analysis_results = analysis_models.predict(pd_valid_dataset,json_file)



In [ ]:
print(analysis_results)

In [ ]:
DataFrame(analysis_results[1]['metrics']['predict']['cm'])

In [ ]:
type(analysis_results[1])

In [40]:
from h2o import H2OFrame  as H2OFrame

ar = h2o.load_model(r'D:/Data/models/PoC-binomial/train/H2OGradientBoostingEstimator_1496088476.7641294')
def convertir_binomial(x):
    if int(x) % 2 == 0:
        return 1
    else:
        return 0
pd_valid_dataset['objective0'] = pd_valid_dataset['objective0'].apply(convertir_binomial)
analysis_results = ar.model_performance(H2OFrame(python_obj=pd_valid_dataset))



Parse progress: |█████████████████████████████████████████████████████████| 100%


In [43]:
for key, value in analysis_results._metric_json.items():
    if not isinstance(value,dict):
        print('parameter: %s - type: %s'% (key, type(value)))

parameter: r2 - type: <class 'float'>
parameter: logloss - type: <class 'float'>
parameter: MSE - type: <class 'float'>
parameter: RMSE - type: <class 'float'>
parameter: predictions - type: <class 'NoneType'>
parameter: thresholds_and_metric_scores - type: <class 'h2o.two_dim_table.H2OTwoDimTable'>
parameter: Gini - type: <class 'float'>
parameter: model_category - type: <class 'str'>
parameter: model_checksum - type: <class 'int'>
parameter: scoring_time - type: <class 'int'>
parameter: AUC - type: <class 'float'>
parameter: domain - type: <class 'list'>
parameter: gains_lift_table - type: <class 'h2o.two_dim_table.H2OTwoDimTable'>
parameter: nobs - type: <class 'int'>
parameter: mean_per_class_error - type: <class 'float'>
parameter: description - type: <class 'NoneType'>
parameter: max_criteria_and_metric_scores - type: <class 'h2o.two_dim_table.H2OTwoDimTable'>
parameter: frame_checksum - type: <class 'int'>


In [ ]:
print(analysis_results._metric_json['max_criteria_and_metric_scores'].as_data_frame())

In [12]:
print(analysis_results._metric_json['max_criteria_and_metric_scores'].as_data_frame().to_json(orient='split'))

{"columns":["metric","threshold","value","idx"],"index":[0,1,2,3,4,5,6,7,8,9],"data":[["max f1",0.3876818651,0.6666666667,399.0],["max f2",0.3876818651,0.8333333333,399.0],["max f0point5",0.3876818651,0.5555555556,399.0],["max accuracy",0.3876818651,0.5,399.0],["max precision",0.3876818651,0.5,399.0],["max recall",0.3876818651,1.0,399.0],["max specificity",0.6183913957,0.999,0.0],["max absolute_mcc",0.5047218476,0.8588068705,196.0],["max min_per_class_accuracy",0.5062693864,0.0708,194.0],["max mean_per_class_accuracy",0.3876818651,0.5,399.0]]}


In [19]:
type(analysis_results.confusion_matrix(metric='accuracy'))

TypeError: confusion_matrix() got an unexpected keyword argument 'metric'

In [32]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
pd_valid_dataset['objective0'] = pd_valid_dataset['objective0'].apply(convertir_binomial)
pd_train_dataset['objective0'] = pd_valid_dataset['objective0'].apply(convertir_binomial)

ar =H2OGradientBoostingEstimator(training_frame=H2OFrame(pd_train_dataset), 
                                 validation_frame=H2OFrame(pd_valid_dataset),
                                 model_id='H2OGradientBoostingEstimator_1496088476.7641294', nfolds=2, 
                                 keep_cross_validation_predictions=True, keep_cross_validation_fold_assignment=True, 
                                 score_each_iteration=True, score_tree_interval=1000, 
                                 fold_assignment='auto', response_column='objective0', 
                                 ignore_const_cols=True, balance_classes=True, 
                                 max_after_balance_size=5.0, max_hit_ratio_k=3, 
                                 ntrees=5, max_depth=10, max_runtime_secs=100, learn_rate=0.05, 
                                 learn_rate_annealing=1.0, distribution='bernoulli')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [34]:
print(analysis_results._metric_json['max_criteria_and_metric_scores'].as_data_frame().to_json(orient='split'))

{"columns":["metric","threshold","value","idx"],"index":[0,1,2,3,4,5,6,7,8,9],"data":[["max f1",0.3876818651,0.6666666667,399.0],["max f2",0.3876818651,0.8333333333,399.0],["max f0point5",0.3876818651,0.5555555556,399.0],["max accuracy",0.3876818651,0.5,399.0],["max precision",0.3876818651,0.5,399.0],["max recall",0.3876818651,1.0,399.0],["max specificity",0.6183913957,0.999,0.0],["max absolute_mcc",0.5047218476,0.8588068705,196.0],["max min_per_class_accuracy",0.5062693864,0.0708,194.0],["max mean_per_class_accuracy",0.3876818651,0.5,399.0]]}


In [39]:
h2o.model_performance(ar,valid=True)

AttributeError: module 'h2o' has no attribute 'model_performance'

In [36]:
anr = ar.model_performance(valid=True)
for key, value in anr._metric_json.items():
    if not isistance(value,dict):
        print('parameter: %s - type: %s'% (parameter, type(value)))
    

TypeError: 'NoneType' object is not subscriptable

In [31]:
ar.__dict__

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 objective0 
 0.468627 0.472549 0.472549 0.476471 0.476471 0.480392 0.480392 0.484314 0.484314 0.488235 0.492157 0.496078 0.496078 0.5 0.5 0.496078 0.460784 0.390196 0.217647 0.00980392 -0.241176 -0.476471 -0.5 -0.492157 -0.488235 -0.5 -0.5 -0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.154902 0.072549 0.131373 0.182353 0.229412 0.386275 0.5 0.5 0.382353 -0.0176471 -0.441176 -0.5 -0.492157 -0.5 -0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.496078 0.5 -0.331373 -0.5 -0.492157 -0.5 -0.5 -0.496078 -0.366667 -0.143137 0.303922 0.5 0.366667 -0.241176 -0.5 -0.488235 -0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.496078 0.5 -0.319608 -0.5 -0.476471 -0.488235 -0.492157 -0.5 -0.5 -0.5 -0.5 -0.154902 0.460784 0.476471 -0.237255 -0.5 -0.488235 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.496078 0.5 -0.331373 -0.5 -0.492157 -0.5 -0.5 -0.496078 -0.492157 -0.484314 -0.484314 -0.5 -0.264706 0.468627 0.394118 -0.421569 -0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 objective0 
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.488235 -0.5 0.103922 0.5 0.484314 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.464706 0.35098 0.162745 -0.107843 -0.441176 -0.5 -0.488235 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.0411765 0.107843 0.178431 0.5 0.496078 0.5 0.194118 0.0843137 0.103922 0.1 0.131373 0.205882 0.366667 0.5 0.496078 0.5 0.370588 -0.147059 -0.5 -0.488235 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.488235 -0.5 -0.139216 0.5 0.5 0.441176 -0.417647 -0.5 -0.496078 -0.5 -0.5 -0.5 -0.472549 -0.264706 0.221569 0.5 0.492157 0.5 0.0294118 -0.5 -0.492157 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.476471 -0.5 0.170588 0.5 0.5 0.22549 -0.5 -0.480392 -0.484314 -0.484314 -0.484314 -0.488235 -0.496078 -0.496078 -0.5 0.0333333 0.5 0.476471 0.5 -0.131373 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.496078 -0.5 -0.445098 0.362745 0.5 0.5 0.00588235 -0.5 -0.484314 -0.5 -0.5 -0.5 -0.5 -0.496078 -0.488235 -0.492157 -0.5 0.166667 0.5 0.5 0.362745 -0.433333 -0.5 -0.5 -0.5 -0.5 -0

{'_bcin': {},
 '_estimator_type': 'unsupervised',
 '_future': False,
 '_id': 'H2OGradientBoostingEstimator_1496088476.7641294',
 '_is_xvalidated': False,
 '_job': None,
 '_metrics_class': None,
 '_model_json': None,
 '_parms': {'balance_classes': True,
  'distribution': 'bernoulli',
  'fold_assignment': 'auto',
  'ignore_const_cols': True,
  'keep_cross_validation_fold_assignment': True,
  'keep_cross_validation_predictions': True,
  'learn_rate': 0.05,
  'learn_rate_annealing': 1.0,
  'max_after_balance_size': 5.0,
  'max_depth': 10,
  'max_hit_ratio_k': 3,
  'max_runtime_secs': 100,
  'model_id': 'H2OGradientBoostingEstimator_1496088476.7641294',
  'nfolds': 2,
  'ntrees': 5,
  'response_column': 'objective0',
  'score_each_iteration': True,
  'score_tree_interval': 1000,
  'training_frame': ,
  'validation_frame': },
 '_xval_keys': None,
 'parms': {}}

In [27]:
from hashlib import md5 as md5
from hashlib import sha256 as sha256
print(hashlib.md5(open(r'D:/Data/models/PoC-binomial/train/H2OGradientBoostingEstimator_1496088476.7641294', 'rb').read()).hexdigest())


c48b85f07fad3d208aa48b4b6ef21925


In [47]:
h2o.cluster().shutdown

<bound method H2OCluster.shutdown of <h2o.backend.cluster.H2OCluster object at 0x000001A9020A6080>>

In [ ]:
valor = {'1':00.1, '2': None, '3': 'cadena'}
try:
    for key, value in valor.items(): 
        print (str(key) +  '-' + str(value))
except:
    print ('Not Found')

In [5]:
h2o.init(nthreads=6)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (Zulu 8.17.0.3-win64) (build 25.102-b14, mixed mode)
  Starting server from D:\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Public\Documents\Wondershare\CreatorTemp\tmphz8xajwe
  JVM stdout: C:\Users\Public\Documents\Wondershare\CreatorTemp\tmphz8xajwe\h2o_e2its_started_from_python.out
  JVM stderr: C:\Users\Public\Documents\Wondershare\CreatorTemp\tmphz8xajwe\h2o_e2its_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,09 secs
H2O cluster version:,3.10.4.6
H2O cluster version age:,"28 days, 10 hours and 11 minutes"
H2O cluster name:,H2O_from_python_e2its_dtf3xx
H2O cluster total nodes:,1
H2O cluster free memory:,3.530 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,6
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None
